# PRONÓSTICO DE DIABETES MEDIANTE EL USO ML

# MODELO DE RED NEURONAL ARTIFICIAL

In [ ]:
#importando librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
import itertools

In [ ]:
#Metodología aplicada según Vásquez Morales, Gabriel Ricardo (2019)
#Gráfica del loss y el accuracy
def plot_acc(history, title="Model Accuracy"):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

def plot_loss(history, title="Model Loss"): 
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper right')
    plt.show()

def plot_compare_losses(history1, history2, name1="Red 1", name2="Red 2", title="Graph title"):
    plt.plot(history1.history['loss'], color="green")
    plt.plot(history1.history['val_loss'], 'r--', color="green")
    plt.plot(history2.history['loss'], color="blue")
    plt.plot(history2.history['val_loss'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 'Train ' + name2, 'Val ' + name2], loc='upper right')
    plt.show()

def plot_compare_accs(history1, history2, name1="Red 1", name2="Red 2", title="Graph title"):
    plt.plot(history1.history['accuracy'], color="green")
    plt.plot(history1.history['val_accuracy'], 'r--', color="green")
    plt.plot(history2.history['accuracy'], color="blue")
    plt.plot(history2.history['val_accuracy'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 'Train ' + name2, 'Val ' + name2], loc='lower right')
    plt.show()


#Grafica matriz de confusión
def plot_cm(cm, classes):
    plt.imshow(cm, cmap=plt.cm.Blues)
    plt.colorbar()
    plt.title('Matriz de confusión Diabetes')
    plt.ylabel("Real")
    plt.xlabel("Predicho")
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.
    
    for i,j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
        plt.text(j,i,format(cf[i,j],'d'),horizontalalignment='center',color='white' if cf[i,j] > thresh else 'black')
    
    plt.show()

In [ ]:
#Montamos drive
# from google.colab import drive
# drive.mount("/content/drive/")

In [ ]:
# lee archivo de datos
data_df = pd.read_csv("Dataset_of_Diabetes_balanceado.csv")
data_df.head()

In [ ]:
# tabla de Diagnosticos
data_df.info()

In [ ]:
#@title
# Transforma variables categóricas

data_df	=	pd.get_dummies(data_df,	columns=['Gender'],	prefix	= ['Sexo'])

data_df['CLASS'][data_df['CLASS']=='N']=0
data_df['CLASS'][data_df['CLASS']=='Y']=1
data_df['CLASS'] = data_df['CLASS'].astype(int)

data_df = data_df[['ID', 'No_Pation', 'AGE', 'Urea',	'Cr', 'HbA1c', 'Chol', 'TG', 'HDL', 'LDL', 'VLDL', 'BMI', 'Sexo_F', 'Sexo_M', 'CLASS']]

data_df.head()

In [ ]:
#  Histograma
data_df[['AGE', 'Urea',	'Cr', 'HbA1c', 'Chol', 'TG', 'HDL', 'LDL', 'VLDL', 'BMI', 'Sexo_F', 'Sexo_M', 'CLASS']].hist(figsize=(15, 12))

In [ ]:
#  sectorizacion de variables de entrada y salida
print('variables de entrada')
X = data_df.iloc[:,2:14]
print(X.shape)

print('variables de salida')
y = data_df.iloc[:,[14]]
print(y.shape)

print()
print('variables de entrada')
X.head()

In [ ]:
# Normaliza datos en X (escala común para rangos variados)
X = (X - X.min()) / (X.max() - X.min())
X

In [ ]:
# Separa datos de entrenamiento y pruebas
X_train, X_test, y_train, y_test = train_test_split( X, y)

# datos de entrenamiento
print('dimensiones de la matriz de entrenamiento')
print(X_train.shape)
print(y_train.shape)
# datos de prueba
print('dimensiones de la matriz de prueba')
print(X_test.shape)
print(y_test.shape)
print()
print('datos de entrenamiento')
X_train

### A) Algoritmo de entrenamiento optimizador SGD

### Función de activación sigmoid

In [ ]:
# Crear red neuronal 
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_dim = X_train.shape[1]))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

# Resumen del modelo
model.summary()

In [ ]:
# compilación del modelo
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

In [ ]:
# Entrenamiento de la red neuronal
history_01 = model.fit(
    X_train.values,
    y_train.values.ravel(),
    validation_split=0.2,
    epochs=80,
    verbose=2
)

In [ ]:

# Obtiene valores predichos para el conjunto de pruebas 
y_pred = model.predict(X_test.values)
y_pred

In [ ]:
# Predicción datos de prueba
y_pred = np.round(y_pred).astype(int)
y_pred

In [ ]:
# Evaluamos la precisión forma 1
acc_score = accuracy_score(y_test, y_pred)
print("Valor de accuracy en conjunto de datos de test, forma 1: ", acc_score)

print('************************************************')
# Evaluamos la precisión forma 2
score = model.evaluate(X_test, y_test)
print("Valor de accuracy en conjunto de datos de test, forma 2: ",score[1])

### Función de activación relu

In [ ]:
# Crear red neuronal 
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = X_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Resumen del modelo
model.summary()

In [ ]:
# compilación del modelo
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

In [ ]:
# Entrenamiento de la red neuronal
history_02 = model.fit(
    X_train.values,
    y_train.values.ravel(),
    validation_split=0.2,
    epochs=80,
    verbose=2
)

In [ ]:
# Obtiene valores predichos para el conjunto de pruebas 
y_pred = model.predict(X_test.values)
y_pred

In [ ]:
# Predicción datos de prueba
y_pred = np.round(y_pred).astype(int)
y_pred

In [ ]:
# Evaluamos la precisión forma 1
acc_score = accuracy_score(y_test, y_pred)
print("Valor de accuracy en conjunto de datos de test, forma 1: ", acc_score)

print('************************************************')
# Evaluamos la precisión forma 2
score = model.evaluate(X_test, y_test)
print("Valor de accuracy en conjunto de datos de test, forma 2: ",score[1])

#### Gráficas de resultados sigmoid vs relu

In [ ]:

plot_loss(history_01, title = "Model Lossx")
plot_compare_losses(history_01, history_02, name1="Sigmoid", name2="Relu", title="Comparación loss con diferentes funciones de activacion")
plot_compare_accs(history_01, history_02, name1="Sigmoid", name2="Relu",	title="Comparación	accuracy	con diferentes funciones de activacion")


### B) Algoritmo de entrenamiento optimizador adam

In [ ]:
# Crear red neuronal 
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = X_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Resumen del modelo
model.summary()

In [ ]:
# compilación del modelo
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Entrenamiento de la red neuronal
history_03 = model.fit(
    X_train.values,
    y_train.values.ravel(),
    validation_split=0.2,
    epochs=80,
    verbose=2
)

In [ ]:
# Obtiene valores predichos para el conjunto de pruebas 
y_pred = model.predict(X_test.values)
y_pred

In [ ]:
# Predicción datos de prueba
y_pred = np.round(y_pred).astype(int)
y_pred

In [ ]:
# Evaluamos la precisión forma 1
acc_score = accuracy_score(y_test, y_pred)
print("Valor de accuracy en conjunto de datos de test, forma 1: ", acc_score)

print('************************************************')
# Evaluamos la precisión forma 2
score = model.evaluate(X_test, y_test)
print("Valor de accuracy en conjunto de datos de test, forma 2: ",score[1])

#### Gráficas de resultados SGD vs ADAM

In [ ]:
plot_compare_losses(history_02, history_03, name1="SGD", name2="Adam", title="Comparación loss con diferentes optimizadores")
plot_compare_accs(history_02, history_03, name1="SGD", name2="Adam",	title="Comparación	accuracy	con diferentes optimizadores")


### C) Regularización (dropout)

In [ ]:
# Crear red neuronal (para sobreajuste, overfitting)
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Resumen del modelo
model.summary()

In [ ]:
# compilación del modelo
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Entrenamiento de la red neuronal
history_04 = model.fit(
    X_train.values,
    y_train.values.ravel(),
    validation_split=0.2,
    epochs=80,
    verbose=2
)

In [ ]:
# Obtiene valores predichos para el conjunto de pruebas 
y_pred = model.predict(X_test.values)
y_pred

In [ ]:
# Predicción datos de prueba
y_pred = np.round(y_pred).astype(int)
y_pred

In [ ]:
# Evaluamos la precisión forma 1
acc_score = accuracy_score(y_test, y_pred)
print("Valor de accuracy en conjunto de datos de test, forma 1: ", acc_score)

print('************************************************')
# Evaluamos la precisión forma 2
score = model.evaluate(X_test, y_test)
print("Valor de accuracy en conjunto de datos de test, forma 2: ",score[1])

#### Gráficas de resultados con regularizacion Dropout

In [ ]:
plot_compare_losses(history_03, history_04, name1="Sin reg", name2="Con	reg",	title="Comparación	loss	con regularización")
plot_compare_accs(history_03, history_04, name1="Sin reg", name2="Con	reg",	title="Comparación	accuracy	con regularización") 

## MODELO FINAL DE RED DE NEURONA ARTIFICIAL

In [ ]:
# Crear red neuronal 
model_final = Sequential()
model_final.add(Dense(64, activation='relu', input_dim = X_train.shape[1]))
model_final.add(Dropout(0.5))
model_final.add(Dense(64, activation='relu'))
model_final.add(Dropout(0.5))
model_final.add(Dense(64, activation='relu'))
model_final.add(Dropout(0.5))
model_final.add(Dense(1, activation='sigmoid'))

# Resumen del modelo
model_final.summary()

In [ ]:
# compilación del modelo
model_final.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
#Agrega early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
# Entrenamiento de la red neuronal
history_final = model_final.fit(
    X_train.values,
    y_train.values.ravel(),
    validation_split=0.2,
    epochs=80,
    verbose=2
)

In [ ]:
# Obtiene valores predichos para el conjunto de pruebas 
y_pred = model_final.predict(X_test.values)
y_pred

In [ ]:
# Predicción datos de prueba
y_pred = np.round(y_pred).astype(int)
y_pred

In [ ]:
# Evaluamos la precisión forma 1
acc_score = accuracy_score(y_test, y_pred)
print("Valor de accuracy en conjunto de datos de test, forma 1: ", acc_score)

print('************************************************')
# Evaluamos la precisión forma 2
score = model.evaluate(X_test, y_test)
print("Valor de accuracy en conjunto de datos de test, forma 2: ",score[1])

#### Gráficas del Modelo final

In [ ]:
plot_loss(history_final, title="Loss de la red neuronal")
plot_acc(history_final, title="Accuracy de la red neuronal")


#### Guarda modelo final

In [ ]:
#Guarda modelo final
# model_final.save('modelo_Red_Neuronal_Diabetes.h5') 


## Evaluación del modelo final de red neuronal

#### Matriz de confusión

In [ ]:
# Obtiene valores predichos para el conjunto de pruebas 
scores = model_final.predict(X_test, verbose=0)
prediccion = np.round(scores).astype(int)

In [ ]:
cf = confusion_matrix(prediccion, y_test)
cf

In [ ]:

plot_cm(cf, ['No = 0', 'Sí = 1'])

#### 1. Precisión, Recall y F1-score

In [ ]:
#Obtiene metricas
tp=cf[1,1]
tn=cf[0,0]
fp=cf[1,0]
fn=cf[0,1]


print("La	metrica	de	sensibilidad	para	el	modelo	es de:",round(tp/(tp+fn),2))
print("La	metrica	de	especificidad	para	el	modelo	es de:",round(tn/(tn+fp),2))

pre=round(tp/(tp+fp),2)

print("La metrica de precision para el modelo es de:",pre)
 
rec=round(tp/(tp+fn),2)

print("La metrica de recall para el modelo es de:",rec)
print("La	metrica	de	f1-score	para	el	modelo	es	de:",round( (2*pre*rec)/(pre+rec),2) )


#### 2. Curva ROC/AUC

In [ ]:
#Obtiene AUC 
auc = roc_auc_score(y_test, scores)
print("El AUC es de:",auc)

#Genera ROC
fpr, tpr, thresholds = roc_curve(y_test, scores)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Curva ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('ROC/AUC')
plt.legend(loc="lower right")
plt.show()

# MODELO CON SVM

In [ ]:
#4.1	Entrenamiento  con SVM

#Realiza entrenamiento con SVM

from sklearn.svm import SVC

# Pruebas del modelo con SVM
clfSVM = SVC(gamma='auto')
clfSVM.fit(X_train, y_train)


#Realiza prediccion en conjunto pruebas con SVM
scores=clfSVM.predict(X_test)
prediccionSVM=np.round(scores).astype(int)
 


## Evaluación del modelo SVM

#### Matriz de confusión

In [ ]:
#Genera matriz de confusión
cf = confusion_matrix(prediccionSVM, y_test) 
cf
plot_cm(cf, ['No = 0', 'Sí = 1'])

#### 1. Precisión, Recall y F1-score

In [ ]:
#4.4	Métricas de evaluación

#Obtiene metricas 
tp=cf[1,1]
tn=cf[0,0]
fp=cf[1,0]
fn=cf[0,1]

print("La	metrica	de	accuracy	para	el	modelo	es de:",round((tp+tn)/(tp+tn+fn+fp),2))
print("La	metrica	de	sensibilidad	para	el	modelo	es de:",round(tp/(tp+fn),2))
print("La	metrica	de	especificidad	para	el	modelo	es de:",round(tn/(tn+fp),2))

pre=round(tp/(tp+fp),2)

print("La metrica de precision para el modelo es de:",pre) 

rec=round(tp/(tp+fn),2)

print("La metrica de recall para el modelo es de:",rec)
print("La	metrica	de	f1-score	para	el	modelo	es	de:",round( (2*pre*rec)/(pre+rec),2) ) 


#### 2. Curva ROC/AUC

In [ ]:
#Obtiene AUC 
auc=roc_auc_score(y_test, scores)
print("El AUC es de:",auc)

#Genera ROC
fpr, tpr, thresholds = roc_curve(y_test, scores)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Curva ROC (AUC = %0.2f)' % auc) 
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('ROC/AUC')
plt.legend(loc="lower right")
plt.show()

# MODELO CON RANDOM FOREST

In [ ]:
# Entrenamiento con Random Forest

from sklearn.ensemble import RandomForestClassifier

#Realiza entrenamiento con Random Forest
clfRF = RandomForestClassifier(n_estimators=10)
clfRF.fit(X_train, y_train)

#Muestra las features mas importantes
importancia = pd.DataFrame(columns=('variable', 'importancia'))
cont=0

for i in X_test.columns: 
  
    importancia.loc[len(importancia)]=[i,clfRF.feature_importances_[cont]] 
    cont=cont+1

importancia=importancia.sort_values(by=['importancia'], ascending=[False]) 
print(importancia[:20])


In [ ]:
#Realiza prediccion en conjunto pruebas con SVM 
scores=clfRF.predict(X_test)
prediccionRF=np.round(scores).astype(int) 
prediccionRF

## Evaluación del modelo Random Forest

#### Matriz de confusión

In [ ]:
#Genera matriz de confusión
cf = confusion_matrix(prediccionRF, y_test) 
cf
plot_cm(cf, ['No = 0', 'Sí = 1'])

#### 1. Precisión, Recall y F1-score

In [ ]:
#Obtiene metricas 
tp=cf[1,1] 
tn=cf[0,0] 
fp=cf[1,0] 
fn=cf[0,1]

print("La	metrica	de	accuracy	para	el	modelo	es de:",round((tp+tn)/(tp+tn+fn+fp),2))
print("La	metrica	de	sensibilidad	para	el	modelo	es de:",round(tp/(tp+fn),2))
print("La	metrica	de	especificidad	para	el	modelo	es de:",round(tn/(tn+fp),2))

pre=round(tp/(tp+fp),2)

print("La metrica de precision para el modelo es de:",pre) 

rec=round(tp/(tp+fn),2)

print("La metrica de recall para el modelo es de:",rec)
print("La	metrica	de	f1-score	para	el	modelo	es	de:",round( (2*pre*rec)/(pre+rec),2) )


#### 2. Curva ROC/AUC

In [ ]:
#Obtiene AUC 
auc=roc_auc_score(y_test, scores)
print("El AUC es de:",auc)

#Genera ROC
fpr, tpr, thresholds = roc_curve(y_test, scores)

plt.figure() 
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Curva ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('ROC/AUC')
plt.legend(loc="lower right")
plt.show()